## scRNA data processing: Part 02
Here we process data from [G. Harmange et al., 2023](https://www.nature.com/articles/s41467-023-41811-8) on WM989 cell line and [A. Janesick et al., 2023](https://www.nature.com/articles/s41467-023-43458-x) on human breast cancer tissue. First, we have done some basic processing based on number of unique genes, number of mitochondrial genes, etc. The code for data processing- prt 01 can be found here. 

In [1]:
# call libraries
import numpy as np
import pandas as pd
import os

In [2]:
PATH = os.path.expanduser('~') + '/Dropbox/suvranil/cellmem'

#### Import data after processing - part 01
In WM989 cell line, there are 7590 cells and 33538 unique raw UMIs in replicate 1. In replicate 02, there are 4941 cells and 33538 unique raw UMIs. In case of human breast tissue data, there are 7967 cells and 36600 unique raw UMIs. 

In [3]:
#importing raw count data after processing part 01
df = pd.read_csv(PATH + '/data/MemSeqData/full_count_mat_batch_1.csv')
# df = pd.read_csv(PATH + '/data/MemSeqData/full_count_mat_batch_1_WM983.csv')
#df = pd.read_csv(PATH + '/data/MemSeqData/full_count_mat_batch_1.csv')
#df = pd.read_csv(PATH + '/data/BreastTissueData/full_count_mat_breast.csv')

#specifying column name
df = df.rename(columns={'Unnamed: 0': 'gene'})
df.set_index('gene', inplace = True)

#### Sparsity reduction: Part 01
We remove genes which have not been expressed in any cells.

In [4]:
# number of nonzero entries in each gene (row).
count_box = df.apply(lambda row: np.count_nonzero(row), axis=1)
df = df[np.array(count_box) > 0]

#### Normalization of gene expression by total raw counts of each cell
We calculate the total raw counts of each cell, and then divide the raw UMI count for each gene in a cell with the total count of that cell. So, instead of raw UMI count of genes in a cell, we work with relative UMI of genes. 

In [5]:
# total raw UMI counts of each cell
col_sum = df.sum(axis = 0)
col_sum = col_sum.astype(np.float64)
#normalization and changing datatype
df = df.divide(col_sum, axis = 1).astype('float32')

#### Sparsity reduction: Part 02
We remove genes which have been expressed in less than 200 cells. 

In [6]:
# number of nonzero entries in each gene (row).
count_box = df.apply(lambda row: np.count_nonzero(row), axis=1)
# selecting genes with greater than 200 nonzero entries.
df = df[np.array(count_box) > 200]
# plt.hist(count_box)

#### Outlier Detection
There are some high jumps in the expression of some genes. We systemically try to identify these kinds of genes and remove them from downstream analysis. 

In [7]:
def Outlier_detection(gene_exp, lower_boundary, upper_boundary):
    nonzero_gene_exp = gene_exp[gene_exp > 0]
    log_nonzero_gene_exp = np.log(nonzero_gene_exp)
    # threshold for (lower_boundary) percentile
    Q1 = np.percentile(log_nonzero_gene_exp, lower_boundary)
    # threshold for (upper_boundary) percentile
    Q5 = np.percentile(log_nonzero_gene_exp, upper_boundary)
    #Interquartile range
    IQR = Q5 - Q1
    #position of lower and upper whiskers
    lw = Q1 - 1.5 * IQR
    uw = Q5 + 1.5 * IQR
    #datapoints below lower whiskers and above upper whiskers
    outliers = log_nonzero_gene_exp[(log_nonzero_gene_exp < lw) | (log_nonzero_gene_exp > uw)]
    return len(outliers)

In [8]:
outlier_arr = np.ones(len(df.index))
#store number of outliers for each gene
for i in range (len(df.index)):
    outlier_arr[i] = Outlier_detection(df.iloc[i].to_numpy(), 9, 91)
#selecting genes with 0 outlier
df = df[outlier_arr == 0]

#### Selecting top variable genes
We select top variable genes by comparing coefficient of variation of each genes.

In [9]:
# caclculating coefficient of variation
cv = df.std(axis = 1) / df.mean(axis = 1)  
# sorting genes in the descending order of CV
df = df.iloc[np.argsort(-cv)]

In [10]:
# save dataframe
df_output = df[: 5000]
# df_output.to_csv(PATH + '/data/MemSeqData' + 
#                  '/HighVariable5000CountMatrix_WM983.csv', 
#                  index=True)